In [1]:
import torch
import os

os.getcwd()

'/ptls-experiments/scenario_rosbank'

In [2]:
# import torch
# import torch.nn as nn

# from torch.fx import symbolic_trace
# import torch.ao.quantization.quantize_fx as quantize_fx
# from typing import List, Dict

# class FXWrapping(nn.Module):
#     def __init__(self, base, qconfig, *args, **kwargs):
#         super().__init__()
#         self.base = base
#         self.example_input = None
#         self.qconfig = qconfig
#         self.args = args
#         self.kwargs = kwargs
#         self._is_prepared = self._is_converted = False

#     def forward(self, *args, **kwargs):
#         # if not isinstance(self.base, nn.RNNBase):
#         #     args = (args[0],)
#         self.example_input = (args, kwargs)
#         return self.base(*args, **kwargs)
    
#     def prepare(self):
#         assert self.example_input is not None, 'Run model with example input!'
#         self.base = quantize_fx.prepare_fx(self.base, self.qconfig, 
#                                            self.example_input[0], **self.example_input[1])
#         self._is_prepared = True
                               
#     def convert(self):
#         assert self._is_prepared, 'Prepare model before convertation!'
#         self.base = quantize_fx.convert_fx(self.base)
#         self._is_converted = True
        

# class FXWrapper:
#     wrappings: Dict[str, FXWrapping] = {'basic': FXWrapping}

#     def __init__(self, qconfig, wrapping='basic', is_qat=False):
#         self.is_qat = is_qat
#         self.qconfig = qconfig
#         self._wrapped_modules: List[FXWrapping] = []
#         self.wrapping = self.wrappings[wrapping]

#     def wrap(self, model: nn.Module, batch_sample):
#         model.eval()
#         self._wrap_init(model)
#         model(batch_sample)
    
#     def _wrap_init(self, model: nn.Module):
#         named_children = list(model.named_children())
#         if not named_children:
#             named_children = [('', model)]
#         for name, child in named_children:
#             trace = self.is_traceable(child)
#             if trace:
#                 wrapped = self.wrapping(child, self.qconfig)
#                 self._wrapped_modules.append(wrapped)
#                 setattr(model, name, wrapped)
#                 print(name, 'name')
#             elif not name:
#                 self._wrap_init(child)

#     def prepare(self):
#         for module in self._wrapped_modules:
#             module.prepare()
    
#     def convert(self):
#         for module in self._wrapped_modules:
#             module.convert()
            
#     @classmethod
#     def is_traceable(cls, module: nn.Module):
#         try:
#             trace = symbolic_trace(module)
#             return trace
#         except:
#             return None
        
# from torch.ao.quantization.qconfig_mapping import QConfigMapping, get_default_qconfig_mapping
# qconfig_mapping = get_default_qconfig_mapping('x86')

# class TestModel(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.lstm = nn.LSTM(10, 11)
#         self.head = nn.Sequential(nn.Linear(11, 1), nn.ReLU())
    
#     def forward(self, x):
#         x, _ = self.lstm(x)
#         x = self.head(x)
#         return x

# batch_sample = torch.randn((19, 7, 10))
# # batch_sample = torch.randn((19, 11))

# model = TestModel()
# wrapper = FXWrapper(qconfig_mapping)
# wrapper.wrap(model, batch_sample)
# wrapper.prepare()
# wrapper.convert()


In [3]:
# wrapper.convert

# Conventional

In [4]:
torch.__version__

'1.13.1+cu117'

In [5]:
def estimate_size(model: torch.nn.Module):
    const = 1
    size = 0
    for module in model.parameters():
        size += module.numel() * module.element_size()
    for buffer in model.buffers():
        size += buffer.numel() * buffer.element_size()
    return size / const

def gain(base, comp):
    base, comp = estimate_size(base), estimate_size(comp)
    return (base - comp) / base * 100

In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Subset

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3 channels
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize for 3 channels
])

# Load MNIST dataset
PATH = 'MNIST'
trainset = torchvision.datasets.MNIST(root=PATH, train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root=PATH, train=False, download=True, transform=transform)

import numpy as np
idx = np.arange(32)
trainset = Subset(trainset, idx)
testset = Subset(testset, idx)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# Load ResNet-50 model
model = models.resnet50(pretrained=True)  # You can use pretrained weights
# Modify the first layer to accept 3 channel input
model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(model.fc.in_features, 10)  # Adjust output for 10 classes (0-9)

# To use the model for training
model.eval()

example_inputs = (next(iter(trainloader))[0])


/root/.local/share/virtualenvs/ptls-experiments-H-SwwRmK/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/.local/share/virtualenvs/ptls-experiments-H-SwwRmK/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
def post_quantize_model(model, qconfig=None, backend='x86'):
    if qconfig is None:
        torch.backends.quantized.engine = backend
        qconfig = torch.quantization.get_default_qconfig(backend)
    model.eval()
    
    model.qconfig = qconfig
    model_quantized = torch.quantization.prepare(torch.ao.quantization.QuantWrapper(model), inplace=False)
    model_quantized = torch.quantization.convert(model_quantized, inplace=True)
    return model_quantized

In [43]:
from inspect import signature

(signature(torch.ao.quantization.quantize).parameters).keys()

odict_keys(['model', 'run_fn', 'run_args', 'mapping', 'inplace'])

In [ ]:
import torch.ao.quantization

placeholder = lambda *args, **kwargs: None

qm = torch.ao.quantization.quantiz(sup_module, placeholder, (1,))

/root/.local/share/virtualenvs/ptls-experiments-H-SwwRmK/lib/python3.8/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/root/.local/share/virtualenvs/ptls-experiments-H-SwwRmK/lib/python3.8/site-packages/torch/ao/quantization/observer.py:1204: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [8]:
qm = post_quantize_model(model)

/root/.local/share/virtualenvs/ptls-experiments-H-SwwRmK/lib/python3.8/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/root/.local/share/virtualenvs/ptls-experiments-H-SwwRmK/lib/python3.8/site-packages/torch/ao/quantization/observer.py:1204: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [9]:
def save_quant(quantized_model, path):
    torch.jit.save(torch.jit.script(quantized_model), path)

def load_quant(path):
    script = torch.jit.load(path)
    return script

In [35]:
gain(model, qm)

99.92875846184091

In [36]:
qm

SequenceToTarget(
  (seq_encoder): RnnSeqEncoder(
    (trx_encoder): TrxEncoder(
      (embeddings): ModuleDict(
        (small_group): NoisyEmbedding(
          150, 32, padding_idx=0
          (dropout): Dropout(p=0, inplace=False)
        )
      )
      (custom_embeddings): ModuleDict(
        (amount_rur): LogScaler()
      )
    )
    (seq_encoder): RnnEncoder(
      (reducer): LastStepEncoder()
      (rnn): GRU(33, 48, batch_first=True)
    )
  )
  (head): Head(
    (model): Sequential(
      (0): Sequential(
        (0): QuantizedLinear(in_features=48, out_features=4, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
        (1): LogSoftmax(dim=1)
      )
    )
  )
  (loss): NLLLoss()
  (train_metrics): ModuleDict(
    (MulticlassAccuracy): MulticlassAccuracy()
  )
  (valid_metrics): ModuleDict(
    (MulticlassAccuracy): MulticlassAccuracy()
  )
  (test_metrics): ModuleDict(
    (MulticlassAccuracy): MulticlassAccuracy()
  )
)

In [ ]:
import inspect
import ast
import re
# print(inspect.getsource(type(model)))
# for name, c in model.named_children():
#     print(name, c)
# (?<substrate>\w+?) ?[=\/\*\+]= ?(?<other>\w+)
print(inspect.getsource(type(model.layer1[0])))
ast.

class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self

## FedCore

In [2]:
from ptls.fedcore_compression.fc_setups import TEST
QUANT = TEST
qkey = 'training_aware_quant'
# qkey = 'post_training_quant'
QUANT['initial_assumption'] = [qkey]
QUANT['model_params'][qkey] = {'epochs': 1}
QUANT['distributed_compression'] = False

QUANT['common']['save_each'] = False



ModuleNotFoundError: No module named 'fc_utils'

In [3]:
QUANT

NameError: name 'QUANT' is not defined

In [16]:
from fedcore.api.main import FedCore
from fedcore.api.utils.data import get_compression_input

compinp = get_compression_input(model, train_dataloader=trainloader, calib_dataloader=trainloader, num_classes=10)
# fcomp = FedCore(compression_task='composite_compression',
#                 need_evo_opt=False,
                
#                 initial_assumption=['training_aware_quant'])
fcomp = FedCore(**QUANT, need_evo_opt=False)

fcomp  

In [17]:
fcomp.fit((compinp, model), manually_done=True)

2024-11-25 16:39:44,746 - Initialising FedCore Repository
2024-11-25 16:39:44,948 - Initialising solver
2024-11-25 16:39:44,950 - Initialising experiment setup


2024-11-25 16:39:45 [INFO] Attention Blocks: 0
2024-11-25 16:39:45 [INFO] FFN Blocks: 0


### PTFXADAPT _get_quantizable_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_recursively
_get_bf16_ops_rec

Batch #: 100%|██████████| 1/1 [00:03<00:00,  3.39s/it]

: 

In [7]:
b = next(iter(testloader))[0]#.to('cuda')

In [8]:
compressed =fcomp.optimised_model.export_compressed_model()

### MODULE QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.012489816173911095, zero_point=0, padding=(3, 3))
### non valid <class 'torch.ao.nn.intrinsic.quantized.modules.conv_relu.ConvReLU2d'>
### MODULE QuantizedConvReLU2d(64, 64, kernel_size=(1, 1), stride=(1, 1), scale=0.00820652861148119, zero_point=0)
### non valid <class 'torch.ao.nn.intrinsic.quantized.modules.conv_relu.ConvReLU2d'>
### MODULE QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.006752930115908384, zero_point=0, padding=(1, 1))
### non valid <class 'torch.ao.nn.intrinsic.quantized.modules.conv_relu.ConvReLU2d'>
### MODULE QuantizedConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), scale=0.014969165436923504, zero_point=148)
### non valid <class 'torch.ao.nn.quantized.modules.conv.Conv2d'>
### MODULE QuantizedConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), scale=0.019215935841202736, zero_point=128)
### non valid <class 'torch.ao.nn.quantized.modules.conv.Conv2d'>
### M

In [9]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
compressed

PyTorchFXModel(
  (_model): GraphModule(
    (conv1): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.013203829526901245, zero_point=0, padding=(3, 3))
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Module(
      (0): Module(
        (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(1, 1), stride=(1, 1), scale=0.0058478922583162785, zero_point=0)
        (conv2): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.007402090821415186, zero_point=0, padding=(1, 1))
        (conv3): QuantizedConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), scale=0.011489166878163815, zero_point=121)
        (downsample): Module(
          (0): QuantizedConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), scale=0.02344454638659954, zero_point=113)
        )
      )
      (1): Module(
        (conv1): QuantizedConvReLU2d(256, 64, kernel_size=(1, 1), stride=(1, 1), scale=0.008299344219267368, zero_point=0)
     

In [11]:
model = torch.load(
    '/ptls-experiments/scenario_rosbank/models/mles_model.p'
)

# PTLS

In [11]:
import torch

import numpy as np
import pandas as pd
import torchmetrics
import pytorch_lightning as pl

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from functools import partial
from ptls.frames import PtlsDataModule
from ptls.nn import TrxEncoder, RnnSeqEncoder, Head
from ptls.data_load.datasets import MemoryMapDataset
from ptls.preprocessing.pandas.pandas_preprocessor import PandasDataPreprocessor
from ptls.frames.supervised import SeqToTargetDataset, SequenceToTarget
from ptls.data_load.utils import collate_feature_dict
from ptls.frames.inference_module import InferenceModule

df_target = pd.read_csv(
    "https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true",
)
df_target.head()

df_target_train, df_target_test = train_test_split(
    df_target, test_size=7000, stratify=df_target["bins"], random_state=142)
df_target_train, df_target_valid = train_test_split(
    df_target_train, test_size=3000, stratify=df_target_train["bins"], random_state=142)
print("Split {} records to train: {}, valid: {}, test: {}".format(
    *[
      len(df)
      for df in [df_target, df_target_train, df_target_valid, df_target_test]
    ]
))

Split 30000 records to train: 20000, valid: 3000, test: 7000


In [12]:
df_trx = pd.read_csv(
    "https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/transactions_train.csv.gz?download=true",
    compression="gzip",
)

df_trx_train = pd.merge(pd.DataFrame(df_trx), df_target_train["client_id"], on="client_id", how="inner")
df_trx_valid = pd.merge(pd.DataFrame(df_trx), df_target_valid["client_id"], on="client_id", how="inner")
df_trx_test = pd.merge(pd.DataFrame(df_trx), df_target_test["client_id"], on="client_id", how="inner")
print("Split {} transactions to train: {}, valid: {}, test: {}".format(
    *[len(df) for df in [df_trx, df_trx_train, df_trx_valid, df_trx_test]]))

Split 26450577 transactions to train: 17622321, valid: 2634248, test: 6194008


In [13]:
preprocessor = PandasDataPreprocessor(
    col_id="client_id",
    col_event_time="trans_date",
    event_time_transformation="none",
    cols_category=["small_group"],
    cols_numerical=["amount_rur"],
    return_records=False,
)
df_data_train = preprocessor.fit_transform(df_trx_train)
df_data_valid = preprocessor.transform(df_trx_valid)
df_data_test = preprocessor.transform(df_trx_test)

print(
    "Record in dataset, train {}, valid {}, test {}".format(
        *[len(df) for df in [df_data_train, df_data_valid, df_data_test]]
    )
)
print("Each record is a client with list of transactions")

df_target = df_target.rename(columns={"bins": "target_bin"})

df_data_train = pd.merge(pd.DataFrame(df_data_train), df_target, on="client_id")
df_data_valid = pd.merge(pd.DataFrame(df_data_valid), df_target, on="client_id")
df_data_test = pd.merge(pd.DataFrame(df_data_test), df_target, on="client_id")

df_data_train = df_data_train.to_dict(orient="records")
df_data_valid = df_data_valid.to_dict(orient="records")
df_data_test = df_data_test.to_dict(orient="records")

# show first 10 transactions from one record
rec = df_data_train[0]
{k: v[:10] if type(v) is torch.Tensor else v for k, v in rec.items()}

dataset_train = MemoryMapDataset(df_data_train)
dataset_valid = MemoryMapDataset(df_data_valid)
dataset_test = MemoryMapDataset(df_data_test)

Creating Dask Server
Link Dask Server - http://10.32.15.89:8787/status
Record in dataset, train 20000, valid 3000, test 7000
Each record is a client with list of transactions


In [14]:
seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(
        embeddings={
            "small_group": {"in": 150, "out": 32},
        },
        numeric_values={
            "amount_rur": "log",
        },
        embeddings_noise=0.001,
    ),
    hidden_size=48,
)
sup_module = SequenceToTarget(
    seq_encoder=seq_encoder,
    head=Head(input_size=seq_encoder.embedding_size, objective="classification", num_classes=4),
    loss=torch.nn.NLLLoss(),
    metric_list=torchmetrics.Accuracy(task="multiclass", num_classes=4),
    optimizer_partial=partial(torch.optim.Adam),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=4, gamma=0.5),
)
sup_data = PtlsDataModule(
    train_data=SeqToTargetDataset(dataset_train, target_col_name="target_bin", target_dtype=torch.long),
    valid_data=SeqToTargetDataset(dataset_valid, target_col_name="target_bin", target_dtype=torch.long),
    test_data=SeqToTargetDataset(dataset_test, target_col_name="target_bin", target_dtype=torch.long),
    train_batch_size=128,
    valid_batch_size=1024,
    train_num_workers=8,
)


In [15]:
example_batch = next(iter(sup_data.train_dataloader()))[0]

In [24]:
sup_module.seq_encoder.trx_encoder.qconfig = None

In [25]:
qm = post_quantize_model(sup_module)

/root/.local/share/virtualenvs/ptls-experiments-H-SwwRmK/lib/python3.8/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/root/.local/share/virtualenvs/ptls-experiments-H-SwwRmK/lib/python3.8/site-packages/torch/ao/quantization/observer.py:1204: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [ ]:
import torch.ao.quantization


class MultipleQuantize(torch.ao.nn.quantized.modules.Quantize)

In [21]:
??qm.quant

Signature:      qm.quant(*input, **kwargs)
Type:           Quantize
String form:    Quantize(scale=tensor([1.]), zero_point=tensor([0]), dtype=torch.quint8)
File:           ~/.local/share/virtualenvs/ptls-experiments-H-SwwRmK/lib/python3.8/site-packages/torch/ao/nn/quantized/modules/__init__.py
Source:        
class Quantize(torch.nn.Module):
    r"""Quantizes an incoming tensor

    Args:
     `scale`: scale of the output Quantized Tensor
     `zero_point`: zero_point of output Quantized Tensor
     `dtype`: data type of output Quantized Tensor
     `factory_kwargs`: Dictionary of kwargs used for configuring initialization
         of internal buffers. Currently, `device` and `dtype` are supported.
         Example: `factory_kwargs={'device': 'cuda', 'dtype': torch.float64}`
         will initialize internal buffers as type `torch.float64` on the current CUDA device.
         Note that `dtype` only applies to floating-point buffers.

    Examples::
        >>> t = torch.tensor([[1., -

In [26]:
qm(example_batch)

TypeError: quantize_per_tensor() received an invalid combination of arguments - got (PaddedBatch, float, int, torch.dtype), but expected one of:
 * (Tensor input, Tensor scale, Tensor zero_point, torch.dtype dtype)
      didn't match because some of the arguments have invalid types: (!PaddedBatch!, !float!, !int!, torch.dtype)
 * (Tensor input, float scale, int zero_point, torch.dtype dtype)
      didn't match because some of the arguments have invalid types: (!PaddedBatch!, float, int, torch.dtype)
 * (tuple of Tensors tensors, Tensor scales, Tensor zero_points, torch.dtype dtype)
      didn't match because some of the arguments have invalid types: (!PaddedBatch!, !float!, !int!, torch.dtype)


In [ ]:
post_quantize_model()

## Compression trial

In [ ]:
torch.__version__

'2.3.1+cu118'

In [40]:
from torch.quantization.quantize_fx import prepare_fx, convert_fx
from torch.ao.quantization import (
    default_dynamic_qconfig, 
    float_qparams_weight_only_qconfig, 
    QConfigMapping, 
    per_channel_dynamic_qconfig,
    float16_static_qconfig
)
from ptls.nn.trx_encoder.noisy_embedding import NoisyEmbedding
# Full docs for supported qconfig for floating point modules/ops can be found in `quantization docs <https://pytorch.org/docs/stable/quantization.html#module-torch.quantization>`_
# Full docs for `QConfigMapping <https://pytorch.org/docs/stable/generated/torch.ao.quantization.qconfig_mapping.QConfigMapping.html#torch.ao.quantization.qconfig_mapping.QConfigMapping>`_
qconfig_mapping = (QConfigMapping()
                   .set_object_type(torch.nn.Embedding, float_qparams_weight_only_qconfig)
                #    .set_global(float_qparams_weight_only_qconfig)
    # .set_object_type(nn.Embedding, float_qparams_weight_only_qconfig)
    # .set_object_type(nn.LSTM, default_dynamic_qconfig)
    # .set_module_name(nn.Conv2d, float16_static_qconfig)
    # .set_object_type(nn.Linear, default_dynamic_qconfig)
    # .set_object_type(TestLinear, float_qparams_weight_only_qconfig)
)
# Load model to create the original model because quantization api changes the model inplace and we want
# to keep the original model for future comparison


model_to_quantize = model

# model_to_quantize.load_state_dict(
#     torch.load(
#         model_data_filepath + 'word_language_model_quantize.pth',
#         map_location=torch.device('cpu')
#         )
#     )

model_to_quantize.eval()

example_inputs = (next(iter(data_module.train_dataloader()))[0])

prepared_model = prepare_fx(model_to_quantize, qconfig_mapping, example_inputs)

# print("prepared model:", prepared_model)
quantized_model = convert_fx(prepared_model)
# print("quantized model", quantized_model)

KeyError: 'arange'

In [ ]:
from fedcore.models.network_impl.layers import IDecomposed

def _recreate_embedding(module):
        assert isinstance(module, torch.nn.Embedding)
        new = torch.nn.Embedding(
            module.num_embeddings,
            module.embedding_dim,
            module.padding_idx,
            module.max_norm,
            module.norm_type,
            module.scale_grad_by_freq,
            module.sparse,
            module.weight,
            getattr(module, 'freeze', None),
            getattr(module.weight, 'device', torch.device('cpu')),
            getattr(module.weight, 'dtype', torch.float32)
        )
        return new

def _recreate_linear(module):
    assert isinstance(module, torch.nn.Linear)
    raise NotImplementedError


class ParentalReassembler:    
    supported_layers = {torch.nn.Embedding: _recreate_embedding,
                        torch.nn.Linear: _recreate_linear}
    
    @classmethod
    def set_layer(cls, model, name, module):
        ### TODO check Sequential
        *path, name = name.split('.')
        current_module = model
        for child in path:
            current_module = getattr(current_module, child)
        setattr(current_module, name, module)
            
    
    @classmethod
    def fetch_module(cls, module):
        is_decomposed = isinstance(module, IDecomposed)
        for supported in cls.supported_layers:
            if isinstance(module, supported):
                return supported, is_decomposed
        return None, is_decomposed
    
    @staticmethod
    def decomposed_handle(*args, **kwargs):
        raise NotImplementedError
    
    @classmethod
    def handle(cls, module, type):
        return cls.supported_layers[type](module)

    @classmethod
    def convert(cls, module):
        associated, is_decomp = cls.fetch_module(module)
        if associated is None:
            print('failed to fetch:', module.__class__.__name__)
            return None
        if is_decomp:
            new_module = cls.decomposed_handle(module)
        else:
            new_module = cls.handle(module, associated)
        return new_module
    
    @classmethod
    def reassemble(cls, model):
        for name, module in model.named_modules():
            print(name)
            new_module = cls.convert(module)
            if new_module is None:
                continue
            cls.set_layer(model, name, new_module)
        return model


ParentalReassembler.reassemble(model)


In [15]:
from ptls.fedcore_compression.fc_setups import TEST
QUANT = TEST
qkey = 'training_aware_quant'
# qkey = 'post_training_quant'
QUANT['initial_assumption'] = [qkey]
QUANT['model_params'][qkey] = {'epochs': 1}
QUANT['distributed_compression'] = False

QUANT['common']['save_each'] = False



In [16]:
from fedcore.api.main import FedCore
from fedcore.api.utils.data import get_compression_input

compinp = get_compression_input(model, train_dataloader=data_module.train_dataloader(), 
                                calib_dataloader=data_module.train_dataloader(), num_classes=10)
# fcomp = FedCore(compression_task='composite_compression',
#                 need_evo_opt=False,
                
#                 initial_assumption=['training_aware_quant'])
fcomp = FedCore(**QUANT, need_evo_opt=False)

fcomp  

In [ ]:
fcomp.fit((compinp, model), manually_done=True)

In [ ]:
from fedcore.neural_compressor.config import PostTrainingQuantConfig
from fedcore.neural_compressor import quantization
conf = PostTrainingQuantConfig(
    # approach="weight_only",
    # op_type_dict={
    #     ".*": {  # re.match
    #         "weight": {
    #             "bits": 8,  # 1-8 bit
    #             "group_size": -1,  # -1 (per-channel)
    #             "scheme": "sym",
    #             "algorithm": "RTN",
    #         },
    #     },
    # },
    # recipes={
    #     # 'rtn_args':{'enable_full_range': True, 'enable_mse_search': True},
    #     # 'gptq_args':{'percdamp': 0.01, 'actorder':True, 'block_size': 128, 'nsamples': 128, 'use_full_length': False},
    #     # 'awq_args':{'enable_auto_scale': True, 'enable_mse_search': True, 'n_blocks': 5},
    # },
)
eval_func = lambda *args, **kwargs: 0.9
q_model = quantization.fit(model, conf, eval_func=eval_func, calib_dataloader=trainloader)
q_model.save("saved_results")
compressed_model = q_model.export_compressed_model()
torch.save(compressed_model, "compressed_model.pt")


In [ ]:
from fedcore.neural_compressor.config import PostTrainingQuantConfig
from fedcore.neural_compressor import quantization
conf = PostTrainingQuantConfig(
    # approach="weight_only",
    # op_type_dict={
    #     ".*": {  # re.match
    #         "weight": {
    #             "bits": 8,  # 1-8 bit
    #             "group_size": -1,  # -1 (per-channel)
    #             "scheme": "sym",
    #             "algorithm": "RTN",
    #         },
    #     },
    # },
    # recipes={
    #     # 'rtn_args':{'enable_full_range': True, 'enable_mse_search': True},
    #     # 'gptq_args':{'percdamp': 0.01, 'actorder':True, 'block_size': 128, 'nsamples': 128, 'use_full_length': False},
    #     # 'awq_args':{'enable_auto_scale': True, 'enable_mse_search': True, 'n_blocks': 5},
    # },
)
eval_func = lambda *args, **kwargs: 0.9
q_model = quantization.fit(model, conf, eval_func=eval_func, calib_dataloader=data_module.train_dataloader())
q_model.save("saved_results")
compressed_model = q_model.export_compressed_model()
torch.save(compressed_model, "compressed_model.pt")
